In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_csv("my_products.csv")
df

,Product Name,Original Price,Rating
0,"POCO C71 (Cool Blue, 128 GB)","₹6,799",4.1
1,"Ai+ Pulse (Blue, 64 GB)","₹5,999",4.3
2,realme P3 Lite 5G Charger in the Box (Midnight...,"₹9,749",4.4
3,"Moto g06 power (Tapestry, 64 GB)","₹7,199",4.2
4,"MOTOROLA g05 (Forest Green, 64 GB)","₹6,999",4.2
...,...,...,...
235,"Ai+ Nova 5G (Black, 128 GB)","₹8,499",4.1
236,"realme Narzo 80 Lite 4G (Beach Gold, 64 GB)","₹7,250",4.3
237,"POCO X7 Pro 5G (Yellow, 256 GB)","₹21,999",4.3
238,"Samsung Galaxy A55 5G (Awesome Navy, 128 GB)","₹24,980",4.4


In [3]:
def clean_data(df):
   
    #Cleans numerical data (Price and Rating).
    
    # Ensure we are working with strings
    df['Original Price'] = df['Original Price'].astype(str)
    
    # Remove '₹' and ','
    df['Original Price'] = df['Original Price'].str.replace('₹', '', regex=False).str.replace(',', '', regex=False)
    
    # Convert to numbers
    df['Original Price'] = pd.to_numeric(df['Original Price'], errors='coerce')
    df['Rating'] = pd.to_numeric(df['Rating'], errors='coerce')
    
    # Drop rows with missing prices (cannot analyze without price)
    df.dropna(subset=['Original Price'], inplace=True)
    
    return df

In [4]:
def extract_features(df):
    
    #Splits 'Product Name' into Model, Color, Storage, RAM, and 5G status.
    # --- HELPER FUNCTION ---
    def parse_name_row(text):
        model = text
        color = "Unknown"
        storage_val = None
        
        match = re.search(r'^(.*?)\s*\((.*)\)', text)
        
        if match:
            raw_model = match.group(1)
            specs_text = match.group(2)
            
            # Clean Model Name
            for junk in ["Charger in the Box", "with No Cost EMI", "5G"]:
                raw_model = raw_model.replace(junk, "")
            model = raw_model.strip()
            
            # Extract Color & Storage
            parts = [s.strip() for s in specs_text.split(',')]
            
            if len(parts) > 0:
                if "GB" not in parts[0]:
                    color = parts[0]
            
            # Find storage
            for part in parts:
                if ("GB" in part or "TB" in part) and ("RAM" not in part):
                    storage_val = part

        return pd.Series([model, color, storage_val])

    # --- MAIN LOGIC ---
    # 1. Apply the helper function
    df[['Model', 'Color', 'Storage']] = df['Product Name'].apply(parse_name_row)
    
    # ⭐ NEW: Extract Brand from Model
    df['Brand'] = df['Model'].str.split().str[0]
    
    # 2. Extract RAM
    df['RAM_GB'] = df['Product Name'].str.extract(r'(\d+)\s*GB RAM', expand=False).astype(float)
    
    # 3. Check for 5G
    df['Is_5G'] = df['Product Name'].str.contains('5G', case=False, na=False)

    # 4. Fallback for missing Storage
    mask = df['Storage'].isna()
    df.loc[mask, 'Storage'] = df.loc[mask, 'Product Name'].str.extract(r'(\d+)\s*GB', expand=False) + " GB"

    return df


In [5]:
try:
    print("--- Starting Data Processing ---")
    
    # 1. Load Data
    df_raw = pd.read_csv('my_products.csv')
    print(f"Loaded {len(df_raw)} rows.")

    # 2. Clean Data
    df_cleaned = clean_data(df_raw)

    # 3. Extract Features
    df_final = extract_features(df_cleaned)

    # 4. Save Final File
    output_filename = 'cleaned_featured_data.csv'
    df_final.to_csv(output_filename, index=False)
    
    print(f"\nSuccess! Processed data saved to: {output_filename}")
    print("\nTop 5 Rows Preview:")
    # Selecting specific columns to check if the split worked
    print(df_final[['Product Name', 'Model', 'Color', 'Storage', 'Original Price']].head())

except FileNotFoundError:
    print("Error: Could not find 'my_products.csv'. Make sure the file exists.")
except KeyError as e:
    print(f"Error: Missing column {e}. Check your CSV headers.")

--- Starting Data Processing ---
Loaded 240 rows.

Success! Processed data saved to: cleaned_featured_data.csv

Top 5 Rows Preview:
                                        Product Name           Model  \
0                       POCO C71 (Cool Blue, 128 GB)        POCO C71   
1                            Ai+ Pulse (Blue, 64 GB)       Ai+ Pulse   
2  realme P3 Lite 5G Charger in the Box (Midnight...  realme P3 Lite   
3                   Moto g06 power (Tapestry, 64 GB)  Moto g06 power   
4                 MOTOROLA g05 (Forest Green, 64 GB)    MOTOROLA g05   

           Color Storage  Original Price  
0      Cool Blue  128 GB            6799  
1           Blue   64 GB            5999  
2  Midnight Lily  128 GB            9749  
3       Tapestry   64 GB            7199  
4   Forest Green   64 GB            6999  


In [6]:
#Again check the dataframe, is it ready to analyze 
df = pd.read_csv('cleaned_featured_data.csv')
df.head(5)

,Product Name,Original Price,Rating,Model,Color,Storage,Brand,RAM_GB,Is_5G
0,"POCO C71 (Cool Blue, 128 GB)",6799,4.1,POCO C71,Cool Blue,128 GB,POCO,NaN,False
1,"Ai+ Pulse (Blue, 64 GB)",5999,4.3,Ai+ Pulse,Blue,64 GB,Ai+,NaN,False
2,realme P3 Lite 5G Charger in the Box (Midnight...,9749,4.4,realme P3 Lite,Midnight Lily,128 GB,realme,NaN,True
3,"Moto g06 power (Tapestry, 64 GB)",7199,4.2,Moto g06 power,Tapestry,64 GB,Moto,NaN,False
4,"MOTOROLA g05 (Forest Green, 64 GB)",6999,4.2,MOTOROLA g05,Forest Green,64 GB,MOTOROLA,NaN,False


In [11]:
# Now let's check the colums with the most null values 
df.isnull().sum()

Product Name        0
Original Price      0
Rating              0
Model               0
Color               0
Storage             0
Brand               0
RAM_GB            240
Is_5G               0
dtype: int64

##### As we can see the column RAM_GB has the most null values so we need to drop it. Also we extract the columns from product name so we should delete this ae well

In [17]:
# drop columns Product Name,RAM_GB
df = df.drop(['Product Name','RAM_GB'], axis=1)

In [18]:
df.head(5)

,Original Price,Rating,Model,Color,Storage,Brand,Is_5G
0,6799,4.1,POCO C71,Cool Blue,128 GB,POCO,False
1,5999,4.3,Ai+ Pulse,Blue,64 GB,Ai+,False
2,9749,4.4,realme P3 Lite,Midnight Lily,128 GB,realme,True
3,7199,4.2,Moto g06 power,Tapestry,64 GB,Moto,False
4,6999,4.2,MOTOROLA g05,Forest Green,64 GB,MOTOROLA,False


In [22]:
# segment mobile phone's pricing 

def price_band(p):
    if p < 15000:
        return "Budget"
    elif p < 30000:
        return "Mid-range"
    elif p < 60000:
        return "Premium"
    else:
        return "Ultra-premium"

df["Price_Band"] = df["Original Price"].apply(price_band)


In [27]:
df.head(5)

,Original Price,Rating,Model,Color,Storage,Brand,Is_5G,Price_Band
0,6799,4.1,POCO C71,Cool Blue,128 GB,POCO,False,Budget
1,5999,4.3,Ai+ Pulse,Blue,64 GB,Ai+,False,Budget
2,9749,4.4,realme P3 Lite,Midnight Lily,128 GB,realme,True,Budget
3,7199,4.2,Moto g06 power,Tapestry,64 GB,Moto,False,Budget
4,6999,4.2,MOTOROLA g05,Forest Green,64 GB,MOTOROLA,False,Budget


In [29]:
# Now lets find value for money Models

df["Value_Score"] = (df["Rating"] / df["Original Price"]) * 10000


In [30]:
df.head(5)

,Original Price,Rating,Model,Color,Storage,Brand,Is_5G,Price_Band,Value_Score
0,6799,4.1,POCO C71,Cool Blue,128 GB,POCO,False,Budget,6.030299
1,5999,4.3,Ai+ Pulse,Blue,64 GB,Ai+,False,Budget,7.167861
2,9749,4.4,realme P3 Lite,Midnight Lily,128 GB,realme,True,Budget,4.513283
3,7199,4.2,Moto g06 power,Tapestry,64 GB,Moto,False,Budget,5.834144
4,6999,4.2,MOTOROLA g05,Forest Green,64 GB,MOTOROLA,False,Budget,6.000857


In [21]:
df.to_csv('cleaned_featured_data.csv', index=False)